<a href="https://colab.research.google.com/github/agirardotti/pyEcov/blob/master/Gu%C3%ADa_de_implementaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Instalar pyRofex

---



In [ ]:
pip install -U https://github.com/agirardotti/pyEcov.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.1 MB/s eta 0:00:00


#Importar pyRofex y otros paquetes

---



In [ ]:
import pyRofex

In [ ]:
from datetime import datetime
import pandas as pd

# Variables Globales

---



Crearemos algunas variables globales que nos permitirán almacenar la MarketData que vayamos recibiendo, así también como los reportes de nuestras órdenes.

In [ ]:
market_data_recibida = []
reporte_de_ordenes = []

# Autenticarse en la API

---





> Consumir método /auth/getToken y almacenar internamente el token . El mismo dura hasta 24 hs y no cambia durante el día.



In [ ]:
pyRofex.initialize(user="",password="",account="",environment=pyRofex.Environment.REMARKET)


# Solicitar la lista de instrumentos disponibles en el mercado

---





> Consumir método /instruments/details y almacenar internamente la información. El listado no suele cambiar durante el día.



In [ ]:
repuesta_listado_instrumento = pyRofex.get_detailed_instruments()
repuesta_listado_instrumento

In [ ]:
listado_instrumentos = repuesta_listado_instrumento['instruments']
tickers_existentes = obtener_array_tickers(listado_instrumentos)

# Validar instrumentos de interés

---





> Si sólo queremos trabajar con algunos instrumentos, debemos validar que los mismos existen en la lista de instrumentos devuelto por la API.



In [ ]:
mis_instrumentos = ["DLR/DIC25", "SOJ.ROS/MAY25","DLR/DIC25", "MERV - XMEV - GGAL - 48hs"]
instrumentos_existentes = validar_existencia_instrumentos(mis_instrumentos,tickers_existentes)
instrumentos_existentes

# Conectarnos a la API WebSocket


> Para poder trabajar en tiempo real con la API, debemos conectarnos previamente a la API WebSocket.



In [ ]:
pyRofex.init_websocket_connection(market_data_handler,order_report_handler,error_handler,exception_error)

NameError: name 'pyRofex' is not defined

# Suscribirse a la MarketData


> Una vez que ya tengamos nuestro listado de instrumentos de interés, y nos hayamos conectado a la API WebSocket, ya podemos suscribirnos vía este protocolo a la MarketData en tiempo real de los instrumentos. De esa manera, nuestro sistema quedará a la escucha de nuevas actualizaciones en el book de precios.



In [ ]:
entries = [pyRofex.MarketDataEntry.BIDS,
           pyRofex.MarketDataEntry.OFFERS,
           pyRofex.MarketDataEntry.LAST]
pyRofex.market_data_subscription(instrumentos_existentes,entries)

# Suscribirse a los Execution Reports


> Antes de enviar una orden al mercado, es necesario que nos suscribamos vía WebSocket a los reportes de ejecución de las órdenes de nuestra cuenta. En estos reportes nos llegará el estado de cada orden que enviemos.



In [ ]:
pyRofex.order_report_subscription(snapshot=True)

# Desarrollar Reporte de Cuentas


> Antes de enviar una orden, es necesario que validemos si nuestra cuenta posee el saldo necesario para que la orden ingrese al  mercado. Debe consultarse el método hasta una vez cada 5 segundos, o antes de enviar cada orden.



In [ ]:
def obtenerSaldoCuenta(cuenta=None):
  resumenCuenta = pyRofex.get_account_report(account=cuenta)
  return resumenCuenta["accountData"]["availableToCollateral"]


# Desarrollar Ruteo de órdenes


> Las órdenes deben enviarse vía WebSocket a través de mensajes del tipo "no" (New Order)



In [ ]:
def comprar(instrumento, cantidad, precio, tipoOrden):
  saldo = obtenerSaldoCuenta()
  if saldo >= cantidad * precio:
    nuevaOrden = pyRofex.send_order(ticker=instrumento,side=pyRofex.Side.BUY,size=cantidad,price=precio,order_type=tipoOrden)
    print("Orden de compra enviada {0}".format(nuevaOrden))
  else:
    print("No hay suficiente saldo para enviar la orden de compra")
    return

def vender(instrumento, cantidad, precio, tipoOrden):
  saldo = obtenerSaldoCuenta()
  if saldo >= cantidad * precio:
    nuevaOrden = pyRofex.send_order(ticker=instrumento,side=pyRofex.Side.SELL,size=cantidad,price=precio,order_type=tipoOrden)
    print("Orden de venta enviada {0}".format(nuevaOrden))
  else:
    print("No hay suficiente saldo para enviar la orden de venta")
    return

#comprar('DLR/ENE23',25,108.25,pyRofex.OrderType.LIMIT)

# Desarrollar cancelación de órdenes


> Es necesario que la cancelación de órdenes esté integrada para que una implementación tenga la posibilidad de cancelar una orden siempre que sea necesario.



In [ ]:
def cancelarOrden(clientId):
  cancel_order = pyRofex.cancel_order(clientId)
  print(cancel_order)

# Handlers


> Desarrollaremos manejadores de eventos para poder procesar la mensajería websocket vayamos recibiendo.



### Procesar la MarketData recibida


> Una vez que nos hayamos suscripto a la MarketData, cada actualización que la API nos envíe, debemos procesarla. Aquí es donde entra en juego el objetivo de nuestra implementación, y qué queremos que nuestro sistema haga cuando recibe un nuevo update de MarketData.



In [ ]:
#Mensaje de MarketData: {'type': 'Md', 'timestamp': 1632505852267, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'DLR/DIC21'}, 'marketData': {'BI': [{'price': 108.25, 'size': 100}], 'LA': {'price': 108.35, 'size': 3, 'date': 1632505612941}, 'OF': [{'price': 108.45, 'size': 500}]}}

def market_data_handler(message):
  print(message)
  ticker = message["instrumentId"]["symbol"]
  bid = message["marketData"]["BI"] if len(message["marketData"]["BI"]) != 0 else [{'price': "-", 'size': "-"}]
  offer = message["marketData"]["OF"] if len(message["marketData"]["OF"]) != 0 else [{'price': "-", 'size': "-"}]
  last = message["marketData"]["LA"]["price"] if message["marketData"]["LA"] != None else 0
  dateLA = message['marketData']['LA']['date'] if message["marketData"]["LA"] != None else 0

  timestamp = message['timestamp']
  objeto_md = {'ticker':ticker,'bid':bid,'offer':offer,'last':last,'dateLA':dateLA,'timestamp':timestamp}
  market_data_recibida.append(objeto_md)
  print("Mensaje de MarketData: {0}".format(message))

### Procesar Execution Reports recibidos

> Cuando nuestra orden se envíe, llegue al mercado, y vaya cambiando de estado, recibiremos un reporte de ejecución con el detalle de la orden y su estado actual. Debemos procesarlo internamente y tomar decisiones si se requiere.

In [ ]:
#{"type":"or","orderReport":{"orderId":"1128056","clOrdId":"user14545967430231","proprietary":"api","execId":"160127155448-fix1-1368","accountId":{"id":"30"},"instrumentId":{"marketId":"ROFX","symbol":"DODic21"},"price":18.000,"orderQty":10,"ordType":"LIMIT","side":"BUY","timeInForce":"DAY","transactTime":"20160204-11:41:54","avgPx":0,"lastPx":0,"lastQty":0,"cumQty":0,"leavesQty":10,"status":"CANCELLED","text":"Reemplazada"}}
def order_report_handler(message):
  print("Mensaje de OrderRouting: {0}".format(message))
  reporte_de_ordenes.append(message)

##Procesar errores o excepciones
> Si recibimos algún error o excepción, tenemos que saber cómo debe interpretarlo nuestro desarrollo.

In [ ]:
def error_handler(message):
  print("Mensaje de error: {0}".format(message))

In [ ]:
def exception_error(message):
  print("Mensaje de excepción: {0}".format(message))

# Funciones útiles

In [ ]:
def obtener_array_tickers(listado):
  listado_final = []
  for inst in listado:
    listado_final.append(inst['instrumentId']['symbol'])
  return listado_final

In [ ]:
def validar_existencia_instrumentos(mi_listado,listado_instrumentos):
  listado_final = []
  for instrumento in mi_listado:
    if instrumento in listado_instrumentos:
      print(f'El instrumento {instrumento} existe en el mercado')
      listado_final.append(instrumento)
    else:
      print(f'El instrumento {instrumento} NO existe en el mercado')
  return listado_final


In [ ]:
def actualizarTablaMD():
  df = pd.DataFrame(columns=pd.Index(['Ticker','Timestamp','Vol. Compra','Precio Compra', 'Precio Venta', 'Vol. Venta', 'Ult. Precio Operado']))
  for md in market_data_recibida:
    df = df.append(
        {
          'Ticker': md['ticker'],
          'Timestamp':datetime.fromtimestamp(int(md['timestamp'])/1000),
          'Vol. Compra':md['bid'][0]['size'],
          'Precio Compra':md['bid'][0]['price'],
          'Precio Venta': md['offer'][0]['price'],
          'Vol. Venta': md['offer'][0]['size'],
          'Ult. Precio Operado': md['last']},
          ignore_index=True)
  return df.style.set_table_styles(
      [{'selector': 'tr:hover',
        'props': [('background-color', 'yellow'),('color', 'black')]},
      {'selector': 'thead',
        'props': [('font-size', '16px'),('padding', '8px'),('background-color','brown')]}]
      )

actualizarTablaMD()


In [ ]:
#{'type': 'or', 'timestamp': 1631140826238, 'orderReport': {'orderId': '220805706', 'clOrdId': 'eMSofNm4oXL92V_s', 'proprietary': 'PBCP', 'execId': '210908063630-fix1-2229680', 'accountId': {'id': 'REM2747'}, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'GGAL/DIC21'}, 'price': 192.0, 'orderQty': 10, 'ordType': 'LIMIT', 'side': 'BUY', 'timeInForce': 'DAY', 'transactTime': '20210908-13:11:04.214-0300', 'avgPx': 0, 'lastPx': 0, 'lastQty': 0, 'cumQty': 0, 'leavesQty': 10, 'status': 'NEW', 'text': 'Aceptada '}}
def actualizarTablaOR():

  df = pd.DataFrame(columns=pd.Index(['Timestamp', 'ID Orden', 'Estado', 'Ticker', 'Precio', 'Cantidad', 'Tipo Orden', 'TIF', 'Cuenta', 'Precio Promedio', 'Cantidad Operada', 'Cantidad Remanente', 'TextoAPI']))
  for order in reporte_de_ordenes:
    if order['orderReport']['status'] != "PENDING_NEW" and order['orderReport']['status'] != "PENDING_CANCEL" :
      df = df.append(
          {
              'Timestamp': datetime.fromtimestamp(int(order['timestamp'])/1000),
              'ID Orden':order['orderReport']['orderId'],
              'Estado':order['orderReport']['status'],
              'Ticker':order['orderReport']['instrumentId']['symbol'],
              'Precio':order['orderReport']['price'],
              'Cantidad':order['orderReport']['orderQty'],
              'Tipo Orden':order['orderReport']['ordType'],
              'TIF': order['orderReport']['timeInForce'],
              'Cuenta': order['orderReport']['accountId']['id'],
              'Precio Promedio': order['orderReport']['avgPx'],
              'Cantidad Operada': order['orderReport']['cumQty'],
              'Cantidad Remanente': order['orderReport']['leavesQty'],
              'TextoAPI': order['orderReport']['text']}, ignore_index=True)
    else:
        df = df.append(
          {
              'Timestamp': datetime.fromtimestamp(int(order['timestamp'])/1000),
              'ID Orden':order['orderReport']['orderId'],
              'Estado':order['orderReport']['status'],
              'Ticker':order['orderReport']['instrumentId']['symbol'],
              'Precio':order['orderReport']['price'],
              'Cantidad':order['orderReport']['orderQty'],
              'Tipo Orden':order['orderReport']['ordType'],
              'TIF': order['orderReport']['timeInForce'],
              'Cuenta': order['orderReport']['accountId']['id'],
              'Precio Promedio': "-",
              'Cantidad Operada': "-",
              'Cantidad Remanente': "-",
              'TextoAPI': order['orderReport']['text']}, ignore_index=True)

  df = df.sort_values(by=['Timestamp'],ascending=False)
  return df

actualizarTablaOR()
